In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from surprise import Reader, Dataset, SVD, SVDpp, NormalPredictor, BaselineOnly, KNNBasic, NMF, accuracy


In [2]:
movies = pd.read_csv('../../Data/ml-latest-small/movies.csv')
ratings = pd.read_csv('../../Data/ml-latest-small/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
#https://www.kaggle.com/cesarcf1977/movielens-data-analysis-beginner-s-first
#much simpler than my attempts...

In [5]:
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [6]:
movies.title.str.split('(')[0][0]

'Toy Story '

In [7]:
thing = movies.title.str.split('(')
movies['short_title'] = [thing[i][0] for i in range(len(thing))]
movies['short_title'] = movies.short_title.str.lstrip().str.rstrip()
movies.short_title

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
9737    Black Butler: Book of the Atlantic
9738                 No Game No Life: Zero
9739                                 Flint
9740          Bungo Stray Dogs: Dead Apple
9741          Andrew Dice Clay: Dice Rules
Name: short_title, Length: 9742, dtype: object

In [8]:
movies.short_title[0]

'Toy Story'

In [9]:
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year # As there are some NaN years, resulting type will be float (decimals)
movies.year

0       1995.0
1       1995.0
2       1995.0
3       1995.0
4       1995.0
         ...  
9737    2017.0
9738    2017.0
9739    2017.0
9740    2018.0
9741    1991.0
Name: year, Length: 9742, dtype: float64

In [10]:
max(movies.year)

2018.0

In [11]:
movies[movies.year.isnull()]

,movieId,title,genres,year,short_title
6059,40697,Babylon 5,Sci-Fi,NaN,Babylon 5
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,NaN,Ready Player One
9091,143410,Hyena Road,(no genres listed),NaN,Hyena Road
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),NaN,The Adventures of Sherlock Holmes and Doctor W...
9179,149334,Nocturnal Animals,Drama|Thriller,NaN,Nocturnal Animals
9259,156605,Paterson,(no genres listed),NaN,Paterson
9367,162414,Moonlight,Drama,NaN,Moonlight
9448,167570,The OA,(no genres listed),NaN,The OA
9514,171495,Cosmos,(no genres listed),NaN,Cosmos
9515,171631,Maria Bamford: Old Baby,(no genres listed),NaN,Maria Bamford: Old Baby


In [12]:
#need to drop movieId 40697, Babylon 5 year NaN
#movieId 140956 year = 2018 idx 9031
#movieId 143410 year = 2015 idx 9091
#149334 year = 2016 9179
#156605 year = 2016 9259
#162414 year = 2016 9367
#171631 year = 2017 9515
#171891 year = 2017 9525
#171749 year = 2016 9518

In [13]:
movies.year.iloc[9031] = 2018
movies.year.iloc[9091] = 2015
movies.year.iloc[9179] = 2016
movies.year.iloc[9259] = 2016
movies.year.iloc[9367] = 2016
movies.year.iloc[9515] = 2017
movies.year.iloc[9525] = 2017
movies.year.iloc[9518] = 2006
movies.year.iloc[9611] = 2011
movies.year.iloc[9138] = 1980
movies.year.iloc[9448] = 2016

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
movies[movies.year.isnull()]

,movieId,title,genres,year,short_title
6059,40697,Babylon 5,Sci-Fi,NaN,Babylon 5
9514,171495,Cosmos,(no genres listed),NaN,Cosmos


In [15]:
movies = movies[~movies.year.isnull()]

In [16]:
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year 
movies.head()

,movieId,title,genres,year,short_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


In [17]:
movies = movies[['movieId', 'title', 'short_title', 'year', 'genres']]

In [18]:
genres_unique = pd.DataFrame(movies.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre']) # Format into DataFrame to store later
movies = movies.join(movies.genres.str.get_dummies().astype(int)) #if you .astype as str? it goes t/f
movies.drop('genres', inplace=True, axis=1)

In [19]:
movies.head()

,movieId,title,short_title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Toy Story,1995,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Jumanji,1995,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Grumpier Old Men,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Waiting to Exhale,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Father of the Bride Part II,1995,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [21]:
popularityTable = ratings.groupby('movieId')['userId'].agg('count').to_frame('popularity')
popularityTable.sort_values('popularity', ascending=False)

,popularity
movieId,
356,329
318,317
296,307
593,279
2571,278
...,...
4093,1
4089,1
58351,1


In [22]:
popularityTable = popularityTable.merge(movies[['movieId', 'title']], left_on='movieId', right_on='movieId')

In [23]:
popularityTable = popularityTable.sort_values('popularity', ascending=False)
popularityTable['ranks'] = [i for i in range(1, len(popularityTable)+1)]

In [24]:
popularityTable = popularityTable[popularityTable.movieId != 40697]
popularityTable = popularityTable[popularityTable.movieId != 171495]

In [25]:
ratings = ratings[ratings.movieId != 40697]
ratings = ratings[ratings.movieId != 171495]

In [26]:
ratings['timestamp'] = [datetime.utcfromtimestamp(i).strftime('%m-%d-%Y') for i in ratings.timestamp]

In [27]:
movies.head()

,movieId,title,short_title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Toy Story,1995,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Jumanji,1995,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Grumpier Old Men,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Waiting to Exhale,1995,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Father of the Bride Part II,1995,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
movies.columns

Index(['movieId', 'title', 'short_title', 'year', '(no genres listed)',
       'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [29]:
movies.drop(['title', 'short_title'], axis=1).iloc[0]

movieId                  1
year                  1995
(no genres listed)       0
Action                   0
Adventure                1
Animation                1
Children                 1
Comedy                   1
Crime                    0
Documentary              0
Drama                    0
Fantasy                  1
Film-Noir                0
Horror                   0
IMAX                     0
Musical                  0
Mystery                  0
Romance                  0
Sci-Fi                   0
Thriller                 0
War                      0
Western                  0
Name: 0, dtype: int64

In [30]:
print ("Number of movies Null values: ", max(movies.isnull().sum()))
print ("Number of ratings Null values: ", max(ratings.isnull().sum()))

Number of movies Null values:  0
Number of ratings Null values:  0


In [31]:
popRating = ratings[['movieId','rating']].groupby('movieId').mean()
popRating = popRating.sort_values('rating', ascending=False)
popRating['ranks'] = [i for i in range(1, len(popRating)+1)]
popRating

,rating,ranks
movieId,,
47736,5.0,1
150554,5.0,2
107951,5.0,3
5607,5.0,4
50999,5.0,5
...,...,...
8632,0.5,9718
125221,0.5,9719
167296,0.5,9720


In [32]:
ratings.groupby('userId')['rating'].agg('count').describe()

count     610.000000
mean      165.298361
std       269.462884
min        20.000000
25%        35.000000
50%        70.500000
75%       168.000000
max      2698.000000
Name: rating, dtype: float64

In [33]:
ratings.groupby('userId')['rating'].agg('mean')

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Name: rating, Length: 610, dtype: float64

In [34]:
ratings[ratings.userId == 3].merge(movies[['movieId', 'short_title']], on='movieId')

,userId,movieId,rating,timestamp,short_title
0,3,31,0.5,05-27-2011,Dangerous Minds
1,3,527,0.5,05-27-2011,Schindler's List
2,3,647,0.5,05-27-2011,Courage Under Fire
3,3,688,0.5,05-27-2011,Operation Dumbo Drop
4,3,720,0.5,05-27-2011,Wallace & Gromit: The Best of Aardman Animation
5,3,849,5.0,05-27-2011,Escape from L.A.
6,3,914,0.5,05-27-2011,My Fair Lady
7,3,1093,0.5,05-27-2011,"Doors, The"
8,3,1124,0.5,05-27-2011,On Golden Pond
9,3,1263,0.5,05-27-2011,"Deer Hunter, The"


In [35]:
len(movies)

9740

In [36]:
recents = movies[movies.year == 2018]

In [37]:
popularityTable

,movieId,popularity,title,ranks
314,356,329,Forrest Gump (1994),1
277,318,317,"Shawshank Redemption, The (1994)",2
257,296,307,Pulp Fiction (1994),3
510,593,279,"Silence of the Lambs, The (1991)",4
1938,2571,278,"Matrix, The (1999)",5
...,...,...,...,...
3053,4093,1,Cop (1988),9718
3049,4089,1,Born in East L.A. (1987),9719
6686,58351,1,City of Men (Cidade dos Homens) (2007),9720
3045,4083,1,Best Seller (1987),9721


In [38]:
popRating = popRating.merge(movies[['movieId', 'title']], 
                            left_on=popRating.index, right_on='movieId').sort_values('rating').set_index('movieId')

In [39]:
os.listdir('../../Data')

['hitrates.csv',
 'ml-latest-small',
 'movies_processed.csv',
 'movie_diversity.csv',
 'popularity_ratings.csv',
 'popularity_table.csv',
 'ratings_processed.csv',
 'user_diversity.csv']

In [40]:
ratings.to_csv('../../Data/ratings_processed.csv')
movies.to_csv('../../Data/movies_processed.csv')
popularityTable.to_csv('../../Data/popularity_table.csv')
popRating.to_csv('../../Data/popularity_ratings.csv')